# Construção da tabela com dados econômicos
Construção de uma tabela a partir da API oficial do Banco Mundial (World Bank Open Data).<br>

**Resumo dos Procedimentos realizados:**

* Carregamento dos indicadores socioeconômicos anuais via api do Banco Mundial.<br>
    Tratamento de caracteres e transformação dos nomes dos países.<br>
    Construção das colunas de `importacao_abs` e `exportacao_abs` a partir de indicadores presentes na tabela.<br>
    Construção da coluna de idh a partir de indicadores presentes na tabela.<br>
    Preenchimento de valores nulos.<br>
    Alteração do nome das colunas do dataset.


## Set up

In [ ]:
# carregamento de bibliotecas
import sys
import wbdata
import pandas as pd
import numpy as np
import datetime
import unidecode
sys.path.append('..')

# funções de tratamento
from src.eda_preprocessing import classificar_idh, obter_dados_idh

## Carregamento dos dados

In [163]:
# carregamento da base de códigos no formato dataframe
tbl_cd_pais = pd.read_csv('../data/external/codigos_paises_wb.csv')

## Pré-Processamento dos dados
Remoção de acento do nome dos paises e construção de uma tabela com os indicadores econômicos anuais disponibilizados pela api.

In [164]:
for coluna in ['country', 'pais_corrigido']:
    tbl_cd_pais[coluna] = tbl_cd_pais[coluna].apply(
        lambda x: unidecode.unidecode(x))

In [165]:
# processo de construção do dataset com indicadores econômicos dos países contidos na tabela tbl_cd_pais através da biblioteca wbdata.

# definição do intervalo de tempo de pesquisa
data_inicio = datetime.datetime(1970, 1, 1)
data_fim = datetime.datetime(2024, 1, 1)

# dicionário contendo os indicadores retornados no dataset
indicadores = {"NY.GDP.MKTP.CD": "PIB total (US$ corrente)",
               "NE.EXP.GNFS.ZS": "Exportações (% do PIB)",
               "NE.IMP.GNFS.ZS": "Importações (% do PIB)",
               "SP.POP.TOTL": "População total",
               "NY.GDP.PCAP.CD": "PIB_per_capita",
               "NY.GDP.MKTP.KD.ZG": "Taxa de crescimento do PIB",
               "FP.CPI.TOTL.ZG": "Taxa de inflação (CPI)",
               "SP.DYN.LE00.IN": "Expectativa de Vida",
               "SE.ADT.LITR.ZS": "Taxa de Alfabetização"}

cod_paises = tbl_cd_pais.codigo_WB.unique().tolist()

# construção do dataset a partir da função get_dataframe
tbl_dados_paises = wbdata.get_dataframe(indicadores, country=cod_paises, date=(data_inicio, data_fim))

# tranformação da tabela multi-index em colunas
tbl_dados_paises.reset_index(inplace=True)

**Transformação da lista de nomes**<br>
Como a Api disponibiliza os dados em inglês, precisamos traduzir o nome dos países para conseguir concatenar esses dados com a base disponível no banco de dados.

In [166]:
traducao_paises = {
    'Aruba': 'Aruba',
    'Afghanistan': 'Afeganistão',
    'Angola': 'Angola',
    'United Arab Emirates': 'Emirados Árabes Unidos',
    'Argentina': 'Argentina',
    'Antigua and Barbuda': 'Antígua e Barbuda',
    'Australia': 'Austrália',
    'Austria': 'Áustria',
    'Belgium': 'Bélgica',
    'Benin': 'Benin',
    'Burkina Faso': 'Burquina Faso',
    'Bangladesh': 'Bangladesh',
    'Bulgaria': 'Bulgária',
    'Bahrain': 'Bahrein',
    'Bahamas, The': 'Bahamas',
    'Bosnia and Herzegovina': 'Bósnia-Herzegovina',
    'Belize': 'Belize',
    'Bermuda':'Bermudas',
    'Bolivia': 'Bolívia',
    'Brazil': 'Brasil',
    'Barbados': 'Barbados',
    'Central African Republic': 'República Centro-Africana',
    'Canada': 'Canadá',
    'Switzerland': 'Suíça',
    'Chile': 'Chile',
    'China': 'China',
    "Cote d'Ivoire": 'Costa do Marfim',
    'Cameroon': 'Camarões',
    'Congo, Rep.': 'Congo',
    'Colombia': 'Colômbia',
    'Comoros': 'Comores',
    'Cabo Verde': 'Cabo Verde',
    'Costa Rica': 'Costa Rica',
    'Coveite (Kuweit)':'Kwait',
    'Cuba': 'Cuba',
    'Cayman Islands': 'Cayman, Ilhas',
    'Curacao': 'Curaçao',
    'Cyprus': 'Chipre',
    'Czechia':'República Tcheca',
    'Germany': 'Alemanha',
    'Djibouti': 'Djibuti',
    'Dominica': 'Dominica',
    'Denmark': 'Dinamarca',
    'Dominican Republic': 'República Dominicana',
    'Algeria': 'Argélia',
    'Ecuador': 'Equador',
    'Egypt, Arab Rep.': 'Egito',
    'Spain': 'Espanha',
    'Estonia': 'Estônia',
    'Finland': 'Finlândia',
    'France': 'França',
    'Gabon': 'Gabão',
    'United Kingdom': 'Reino Unido',
    'Germany': 'Alemanha, República Democrática',
    'Georgia': 'Geórgia',
    'Ghana': 'Gana',
    'Equatorial Guinea': 'Guiné Equatorial',
    'Greece': 'Grécia',
    'Grenada': 'Granada',
    'Guatemala': 'Guatemala',
    'Guyana': 'Guiana',
    'Guinea-Bissau': 'Guiné-Bissau',
    'Hong Kong SAR, China': 'Hong Kong',
    'Honduras': 'Honduras',
    'Croatia': 'Croácia',
    'Haiti': 'Haiti',
    'Hungary': 'Hungria',
    'Indonesia': 'Indonésia',
    'India': 'Índia',
    'Ireland': 'Irlanda',
    'Iran, Islamic Rep.': 'Irã',
    'Iraq': 'Iraque',
    'Iceland': 'Islândia',
    'Israel': 'Israel',
    'Italy': 'Itália',
    'Jamaica': 'Jamaica',
    'Jordan': 'Jordânia',
    'Japan': 'Japão',
    'Kenya': 'Quênia',
    'Kyrgyz Republic': 'Quirguistão',
    'St. Kitts and Nevis': 'São Cristóvão e Nevis',
    'Korea, Rep.': 'Coreia, Republica Sul',
    'Kuwait':'Kuwait',
    'Lebanon': 'Líbano',
    'Liberia': 'Libéria',
    'Libya': 'Líbia',
    'Sri Lanka': 'Sri Lanka',
    'Lithuania': 'Lituânia',
    'Luxembourg': 'Luxemburgo',
    'Latvia': 'Letônia',
    'Macao SAR, China': 'Macau',
    'Maldives': 'Maldivas',
    'Morocco': 'Marrocos',
    'Monaco': 'Mônaco',
    'Mexico': 'México',
    'North Macedonia': 'Macedônia do Norte',
    'Malta': 'Malta',
    'Montenegro': 'Montenegro',
    'Mongolia': 'Mongólia',
    'Mozambique': 'Moçambique',
    'Mauritania': 'Mauritânia',
    'Mauritius': 'Maurício',
    'Malawi': 'Malavi',
    'Malaysia': 'Malásia',
    'Namibia': 'Namíbia',
    'Nigeria': 'Nigéria',
    'Nicaragua': 'Nicarágua',
    'Netherlands': 'Países Baixos',
    'Norway': 'Noruega',
    'New Zealand': 'Nova Zelândia',
    'Oman': 'Omã',
    'Pakistan': 'Paquistão',
    'Panama': 'Panamá',
    'Peru': 'Peru',
    'Philippines': 'Filipinas',
    'Palau': 'Palau',
    'Poland': 'Polônia',
    'Puerto Rico': 'Porto Rico',
    "Korea, Dem. People's Rep.": 'Coreia do Norte',
    'Portugal': 'Portugal',
    'Paraguay': 'Paraguai',
    'Qatar': 'Catar',
    'Romania': 'Romênia',
    'Russian Federation': 'Rússia, Federação',
    'Saudi Arabia': 'Arábia Saudita',
    'Senegal': 'Senegal',
    'Singapore': 'Singapura',
    'Sierra Leone': 'Serra Leoa',
    'El Salvador': 'El Salvador',
    'Serbia': 'Sérvia',
    'Sao Tome and Principe': 'São Tomé e Príncipe',
    'Suriname': 'Suriname',
    'Slovenia': 'Eslovênia',
    'Slovak Republic': 'Eslovaca, República',
    'Sweden': 'Suécia',
    'Sint Maarten (Dutch part)': 'Sint Maarten',
    'Eswatini': 'Suazilândia',
    'Togo': 'Togo',
    'Thailand': 'Tailândia',
    'Trinidad and Tobago': 'Trinidade e Tobago',
    'Tunisia': 'Tunísia',
    'Turkiye': 'Turquia',
    'Tuvalu': 'Tuvalu',
    'Tanzania': 'Tanzânia',
    'Uruguay': 'Uruguai',
    'United States': 'Estados Unidos',
    'St. Vincent and the Grenadines': 'São Vicente e Granadinas',
    'Venezuela, RB': 'Venezuela',
    'Virgin Islands (U.S.)': 'Ilhas Virgens Americanas',
    'Viet Nam': 'Vietnã',
    'Vanuatu': 'Vanuatu',
    'South Africa': 'África do Sul'
}

In [ ]:
# verificação de valores únicos
tbl_dados_paises.country.unique()

array(['Aruba', 'Afghanistan', 'Angola', 'United Arab Emirates',
       'Argentina', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Belgium', 'Benin', 'Bangladesh', 'Bulgaria', 'Bahrain',
       'Bahamas, The', 'Bosnia and Herzegovina', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Rep.', 'Colombia',
       'Comoros', 'Cabo Verde', 'Costa Rica', 'Cuba', 'Cayman Islands',
       'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Spain', 'Estonia',
       'Finland', 'France', 'Gabon', 'United Kingdom', 'Ghana',
       'Guinea-Bissau', 'Equatorial Guinea', 'Greece', 'Grenada',
       'Guatemala', 'Guyana', 'Hong Kong SAR, China', 'Honduras',
       'Croatia', 'Haiti', 'Hungary', 'Indonesia', 'India', 'Ireland',
       'Iran, Islamic Rep.', 'Iraq', 'Iceland', 'Italy', 'Jamaica',
       'Jordan', 'Japan', 'Kenya', '

In [ ]:
#transformação dos valores da coluna country
tbl_dados_paises['country'] = tbl_dados_paises['country'].map(traducao_paises).apply(lambda x: unidecode.unidecode(x))

In [169]:
# visualização dos dados antes do pre-processamento
tbl_dados_paises = tbl_dados_paises.merge(tbl_cd_pais, on='country', how='left')
tbl_dados_paises

,country,date,PIB total (US$ corrente),Exportações (% do PIB),Importações (% do PIB),População total,PIB_per_capita,Taxa de crescimento do PIB,Taxa de inflação (CPI),Expectativa de Vida,Taxa de Alfabetização,pais_corrigido,codigo_WB
0,Aruba,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aruba,ABW
1,Aruba,2023,3.648573e+09,88.343786,81.454497,107359.0,33984.790620,4.263719,NaN,76.353,NaN,Aruba,ABW
2,Aruba,2022,3.279344e+09,87.570280,85.313891,107310.0,30559.533535,8.517918,NaN,76.226,NaN,Aruba,ABW
3,Aruba,2021,2.929447e+09,70.740043,76.275533,107700.0,27200.061079,24.132627,NaN,73.655,NaN,Aruba,ABW
4,Aruba,2020,2.481857e+09,52.992604,71.144662,108587.0,22855.932320,-26.211821,NaN,75.406,97.989998,Aruba,ABW
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7695,Africa do Sul,1972,2.451592e+10,21.458134,19.314443,23609456.0,1038.394075,1.654830,6.425676,56.377,NaN,Africa do Sul,ZAF
7696,Africa do Sul,1971,2.341108e+10,18.478261,22.019828,22942143.0,1020.439836,4.278934,5.957421,56.020,NaN,Africa do Sul,ZAF
7697,Africa do Sul,1971,2.341108e+10,18.478261,22.019828,22942143.0,1020.439836,4.278934,5.957421,56.020,NaN,Africa do Sul,ZAF
7698,Africa do Sul,1970,2.121839e+10,18.401953,21.384270,22279984.0,952.352188,5.248661,4.991925,55.681,NaN,Africa do Sul,ZAF


In [ ]:
# remoção de acentos e caracteres especiais.
tbl_dados_paises['country'] = tbl_dados_paises['country'].apply(lambda x: unidecode.unidecode(x))

In [171]:
# construção das colunas de valor absoluto de importação e exportação a partir da colunas que representama % do PIB
tbl_dados_paises['exportacao_abs'] = tbl_dados_paises['PIB total (US$ corrente)'] * \
    tbl_dados_paises['Exportações (% do PIB)']
tbl_dados_paises['importacao_abs'] = tbl_dados_paises['PIB total (US$ corrente)'] * \
    tbl_dados_paises['Importações (% do PIB)']

In [172]:
# construção da coluna idh a partir da função gerar_coluna_idh
tbl_dados_paises = obter_dados_idh(tbl_dados_paises)

In [173]:
# preenchimento dos valores nulos do dataset

# lista das colunas que serão preenchidas pelo método bfill
lista = ['PIB total (US$ corrente)', 'Exportações (% do PIB)',
         'Importações (% do PIB)', 'População total', 'PIB_per_capita',
         'Taxa de crescimento do PIB', 'Taxa de inflação (CPI)',
         'Expectativa de Vida', 'Taxa de Alfabetização', 'importacao_abs', 'exportacao_abs','IDH_estimado']

# aplicação do método bfill para preenchimento de valores nulos em cada grupo.
tbl_dados_paises[lista] = tbl_dados_paises.groupby(
    'country', as_index=False, dropna=False)[lista].bfill()

In [174]:
# preenchimento da coluna taxa alfabetização com o valor padrão de 95, pois valores nulos indicam um índice superior.
tbl_dados_paises['Taxa de Alfabetização'] = tbl_dados_paises['Taxa de Alfabetização'].fillna(value=95)


In [175]:
tbl_dados_paises.drop(columns=['idx_vida','idx_educ','idx_renda','Exportações (% do PIB)','Importações (% do PIB)'], inplace=True)

In [178]:
# correção do nome das colunas para exportação do dataset
tbl_dados_paises = tbl_dados_paises.rename(columns = {'PIB total (US$ corrente)':'PIB_total_(dolar_corrente)', 
                                  'População total':'população_total',
                                  'Taxa de crescimento do PIB':'taxa_crescimento_PIB',
                                  'Taxa de inflação (CPI)':'taxa_inflação_(CPI)',
                                  'Expectativa de Vida':'expectativa_vida',
                                  'Taxa de Alfabetização':'taxa_alfabetização',
                                  'Codigo_WB':'codigo_WB',
                                  })

In [179]:
tbl_dados_paises.head()

,country,date,PIB_total_(dolar_corrente),população_total,PIB_per_capita,taxa_crescimento_PIB,taxa_inflação_(CPI),expectativa_vida,taxa_alfabetização,pais_corrigido,codigo_WB,exportacao_abs,importacao_abs,IDH_estimado
0,Aruba,2024,3.648573e+09,107359.0,33984.790620,4.263719,4.257462,76.353,97.989998,Aruba,ABW,3.223288e+11,2.971927e+11,0.881662
1,Aruba,2023,3.648573e+09,107359.0,33984.790620,4.263719,4.257462,76.353,97.989998,Aruba,ABW,3.223288e+11,2.971927e+11,0.881662
2,Aruba,2022,3.279344e+09,107310.0,30559.533535,8.517918,4.257462,76.226,97.989998,Aruba,ABW,2.871730e+11,2.797736e+11,0.881662
3,Aruba,2021,2.929447e+09,107700.0,27200.061079,24.132627,4.257462,73.655,97.989998,Aruba,ABW,2.072292e+11,2.234451e+11,0.881662
4,Aruba,2020,2.481857e+09,108587.0,22855.932320,-26.211821,4.257462,75.406,97.989998,Aruba,ABW,1.315201e+11,1.765709e+11,0.881662


In [180]:
# salvar o dataset para uso na etapa pre-processamento dos dados.
tbl_dados_paises.to_csv(
    '../data/processed/indicadores_economicos_processed.csv', index=False)